In [ ]:
from src.custom_agents.planner_agent import PlannerAgent

planner_agent = PlannerAgent()


In [ ]:
await planner_agent.execute("Make a SWOT analysis for apple Inc.")

In [ ]:
from src.custom_agents.base import BaseAgent
import os 
agent = BaseAgent("test bot", "", os.getenv(""))
await agent.execute("Make a SWOT analysis for apple Inc.")

In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI, AsyncAzureOpenAI

_ = load_dotenv()

client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

In [ ]:
from agents import Agent, Runner, OpenAIChatCompletionsModel

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant",
    model=OpenAIChatCompletionsModel(
        model=os.getenv("GPT4O_MINI_DEPLOYMENT"),
        openai_client=client,
    )
)

In [ ]:
result = await Runner.run(
    agent,
    "Make a SWOT analysis for apple Inc.",
)

In [ ]:
result.raw_responses

In [ ]:
from custom_agents.swot_agent import SWOTAgent

planner_agent = SWOTAgent()
resutl = await planner_agent.execute("Make a SWOT analysis for apple Inc.")

In [ ]:
for output in resutl.final_output:
    print(output)

In [ ]:
from custom_agents.base_agent import BaseAgent

base_agent = BaseAgent(
    name="Chatbot",
)

In [ ]:
result = await base_agent.execute("Wie alt wurde Albert Einstein am 14. März 1879?")

In [ ]:
result.final_output

In [ ]:
# from dotenv import load_dotenv
# import os
from manager import SWOTAnalysisManager

# load_dotenv()

# print(os.getenv("O3_MINI_DEPLOYMENT"))

swot_manager = SWOTAnalysisManager()

In [ ]:
result = await swot_manager.run("Make a very detailed SWOT analysis for SAP and use data from 2025.")

In [ ]:
result

In [ ]:
from custom_agents.config import Settings

settings = Settings()